### today
- confidently code with OpenAI Chat Completions API
- use one-shot prompting, streaming, markdown, and json results 
- implement a business solution

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

### company sales brochure 
- use open AI API
- use one shot prompting 

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-5-nano'
openai = OpenAI()

API key looks good so far


In [3]:
# https://www.goodreads.com/about/us
# https://www.goodreads.com/
links = fetch_website_links("https://www.goodreads.com/")
links

['/',
 'https://www.goodreads.com/ap/signin?identityProvider=LoginWithAmazon&language=en_US&openid.assoc_handle=amzn_goodreads_web_lwa&openid.claimed_id=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0%2Fidentifier_select&openid.identity=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0%2Fidentifier_select&openid.mode=checkid_setup&openid.ns=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0&openid.pape.max_auth_age=0&openid.return_to=https%3A%2F%2Fwww.goodreads.com%2Fap-handler%2Fsign-in%3FIDP%3DLoginWithAmazon&relyingParty=GOODREADS&siteState=eyJyZXR1cm5fdXJsIjoiaHR0cHM6Ly93d3cuZ29vZHJlYWRzLmNvbS8ifQ%3D%3D',
 'https://www.goodreads.com/ap/signin?identityProvider=APPLE&language=en_US&openid.assoc_handle=amzn_goodreads_web_apple&openid.claimed_id=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0%2Fidentifier_select&openid.identity=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0%2Fidentifier_select&openid.mode=checkid_setup&openid.ns=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0&openid.pape.max_auth_age=0&openid.return_to

In [4]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [5]:

def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [6]:
print(get_links_user_prompt("https://goodreads.com"))


Here is the list of links on the website https://goodreads.com -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

/
https://www.goodreads.com/ap/signin?identityProvider=LoginWithAmazon&language=en_US&openid.assoc_handle=amzn_goodreads_web_lwa&openid.claimed_id=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0%2Fidentifier_select&openid.identity=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0%2Fidentifier_select&openid.mode=checkid_setup&openid.ns=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0&openid.pape.max_auth_age=0&openid.return_to=https%3A%2F%2Fwww.goodreads.com%2Fap-handler%2Fsign-in%3FIDP%3DLoginWithAmazon&relyingParty=GOODREADS&siteState=eyJyZXR1cm5fdXJsIjoiaHR0cHM6Ly93d3cuZ29vZHJlYWRzLmNvbS8ifQ%3D%3D
https://www.goodreads.com/ap/signin?identityProvider=APPLE&language=en_US&openid.assoc_handle=amzn_goodreads_web_ap

In [7]:
def select_relevant_links(url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    return links
    

In [8]:
select_relevant_links("https://goodreads.com")

{'links': [{'type': 'about page', 'url': 'https://www.goodreads.com/about/us'},
  {'type': 'careers page', 'url': 'https://www.goodreads.com/jobs'},
  {'type': 'advertisers page', 'url': 'https://www.goodreads.com/advertisers'},
  {'type': 'author program page',
   'url': 'https://www.goodreads.com/author/program'},
  {'type': 'linkedin page',
   'url': 'https://www.linkedin.com/company/goodreads-com/'},
  {'type': 'facebook page', 'url': 'https://www.facebook.com/Goodreads/'},
  {'type': 'twitter profile', 'url': 'https://twitter.com/goodreads'},
  {'type': 'instagram profile',
   'url': 'https://www.instagram.com/goodreads/'}]}

In [9]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [11]:
print(fetch_page_and_all_relevant_links("https://goodreads.com"))

## Landing Page:

Goodreads | Meet your next favorite book

Discover & read more
Continue with Amazon
Continue with Apple
Sign up with email
By creating an account, you agree to the Goodreads
Terms of Service
and
Privacy Policy
.
Already a member?
Sign In
Deciding what to read next?
You’re in the right place. Tell us what titles or genres you’ve enjoyed in the past, and we’ll give you surprisingly insightful recommendations.
What are your friends reading?
Chances are your friends are discussing their favorite
      (and least favorite) books on Goodreads.
What will
you
discover?
Because Shomeret liked…
She discovered:
Historical Fiction
Because Deborah liked…
She discovered:
Historical Fiction
Search and browse books
Art
Biography
Business
Children's
Christian
Classics
Comics
Cookbooks
Ebooks
Fantasy
Fiction
Graphic Novels
Historical Fiction
History
Horror
Memoir
Music
Mystery
Nonfiction
Poetry
Psychology
Romance
Science
Science Fiction
Self Help
Sports
Thriller
Travel
Young Adult
More

In [12]:
brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""


In [13]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [16]:
get_brochure_user_prompt("Goodreads", "https://goodreads.com")

"\nYou are looking at a company called: Goodreads\nHere are the contents of its landing page and other relevant pages;\nuse this information to build a short brochure of the company in markdown without code blocks.\n\n\n## Landing Page:\n\nGoodreads | Meet your next favorite book\n\nDiscover & read more\nContinue with Amazon\nContinue with Apple\nSign up with email\nBy creating an account, you agree to the Goodreads\nTerms of Service\nand\nPrivacy Policy\n.\nAlready a member?\nSign In\nDeciding what to read next?\nYou’re in the right place. Tell us what titles or genres you’ve enjoyed in the past, and we’ll give you surprisingly insightful recommendations.\nWhat are your friends reading?\nChances are your friends are discussing their favorite\n      (and least favorite) books on Goodreads.\nWhat will\nyou\ndiscover?\nBecause Brian liked…\nHe discovered:\nNonfiction, History\nBecause Brian liked…\nHe discovered:\nYA, Dystopia, Romance\nSearch and browse books\nArt\nBiography\nBusiness\n

In [17]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [18]:
create_brochure("Goodreads", "https://goodreads.com")

# Goodreads Brochure

---

## Meet Goodreads  
**Tagline:** Meet your next favorite book  

Goodreads is the world’s largest site for readers and book recommendations. Launched in January 2007, our mission is simple yet powerful: **to help readers discover books they love and get more out of reading.** Whether you're looking for your next captivating novel or a deep dive into nonfiction, Goodreads connects you with books tailored specifically to your tastes.

---

## What You Can Do on Goodreads  
- **Discover Books:** Get surprisingly insightful recommendations based on your favorite titles and genres.  
- **Social Reading:** See what your friends are reading and join the conversations around your favorite (and least favorite) books.  
- **Track Your Reading:** Keep track of books you're currently reading, have read, and want to read.  
- **Personalized Recommendations:** Powered by analyzing 20 billion data points, our recommendation engine suggests books that align with your unique literary interests.  
- **Community Reviews:** Learn whether a book is right for you by browsing thousands of member reviews.  
- **Explore a Wide Array of Genres:** From Fantasy to Science Fiction, Memoir to Mystery, Poetry to Self-Help—there’s something for every reader.

---

## Company Culture & Values  
- **Passionate About Books & Readers:** At Goodreads, we believe the right book at the right time can change the world.  
- **Community Driven:** We provide a vibrant platform where book lovers come together to share opinions, quotes, and reading challenges.  
- **Innovative & Data-Driven:** Our recommendation system is powered by cutting-edge data analysis to personalize reading experiences.  
- **Inclusive & Diverse:** With genres spanning art, history, romance, young adult, and more, Goodreads celebrates all forms of literature and all types of readers.

---

## Careers at Goodreads  
Goodreads offers exciting career opportunities for those passionate about books, technology, and community-building. Working here means:  
- Joining a dedicated and inclusive team that values innovation and reader happiness.  
- Contributing to one of the largest reading communities worldwide.  
- Helping to transform the way people discover and discuss books every day.  
- Roles span technology, community engagement, data analytics, and content management.  

To learn more and apply, visit our Careers page on Goodreads.

---

## Who Joins Goodreads?  
- **Readers & Book Lovers:** From casual readers searching for their next light read to voracious bibliophiles hunting rare gems.  
- **Authors & Publishers:** Engaging directly with one of the biggest reading communities to promote books and connect with fans.  
- **Advertisers:** Looking to reach an audience deeply invested in literature.  
- **Tech and Data Experts:** Innovators who enhance the user experience with state-of-the-art recommendation algorithms.

---

## Join Us & Discover Your Next Favorite Book  
Goodreads is more than a website—it’s a global community united by a shared love of reading. Whether you want to explore new genres, track your reading journey, participate in challenges, or share reviews and quotes, Goodreads is your literary home.

**Sign up today and transform how you read and discover books!**

---

**Follow Us:**  
- Twitter: @goodreads  
- Facebook: /goodreads  
- Instagram: /goodreads  

Visit us at [goodreads.com](https://www.goodreads.com)  

---

*“The right book in the right hands at the right time can change the world.”*  
— Goodreads Mission Statement

In [19]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [20]:
stream_brochure("Goodreads", "https://goodreads.com")

# Goodreads Brochure

---

## Meet Your Next Favorite Book with Goodreads

**Discover & Read More**  
Goodreads is the world’s largest community for readers and book recommendations. Whether you want to find your next great read, connect with friends about books, or track your personal reading journey, Goodreads is the perfect place to dive deeper into literature.

---

## About Goodreads

### Our Mission  
*To help readers discover books they love and get more out of reading.*  
Launched in January 2007, Goodreads has grown into the go-to platform for book lovers worldwide to explore new genres, share reviews, and connect with fellow readers.

### What You Can Do on Goodreads:  
- See what books your friends are reading and sharing opinions about  
- Track the books you’re currently reading, have read, or want to read  
- Get personalized book recommendations powered by analyzing over 20 billion data points tailored to your unique tastes  
- Read reviews and ratings from a large, diverse community to decide if a book is right for you  

### Book Genres Explored  
From Art to Young Adult, Goodreads covers a vast spectrum including:  
Art | Biography | Business | Children’s | Classics | Fantasy | Fiction | Graphic Novels | History | Memoir | Mystery | Nonfiction | Poetry | Psychology | Romance | Science Fiction | Self Help | Thrillers | and many more!

### Inspiring Quotes  
Goodreads celebrates the love of books and includes a rich collection of inspiring literary quotes from iconic authors like Oscar Wilde, Marilyn Monroe, Albert Einstein, and Dr. Seuss to motivate and delight its community.

---

## Community & Culture

Goodreads values connection through books. The platform encourages lively discussions among readers, sharing honest and heartfelt reviews, and supporting diverse voices across all genres. By fostering a welcoming environment, Goodreads creates a global space where readers can freely explore their passion and discover new perspectives.

---

## Careers at Goodreads

Join a passionate team dedicated to transforming how the world reads. Goodreads offers career opportunities for those eager to innovate in the literary and tech spaces. While specific job listings vary, the company prides itself on a work culture centered around creativity, collaboration, and the shared love of books.

If you are a book lover with a passion for technology and community building, Goodreads could be your ideal workplace to help readers everywhere find the right book at the right time.

---

## Connect With Goodreads

- Website: [goodreads.com](https://www.goodreads.com)  
- Social Media: /goodreads (Facebook, Instagram, Twitter)  
- Follow for updates, giveaways, author events, and more!

---

## Why Goodreads?

*The right book in the right hands at the right time can change the world.*

Whether you are a casual reader or an avid bookworm, Goodreads offers a personalized, engaging, and supportive environment for exploring literature. Join millions of readers and start your next reading adventure today.

---

**Goodreads**  
*Meet your next favorite book.*